Imports

In [39]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import math
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
import numpy as np

Read Datafile

In [40]:
file = open("./data/SMSSpamCollection.txt")
labels=[]
sms=[]
while True:
	content=file.readline().lower()
	if not content:
		break
	sms.append(content[4:].replace("\n", ""))
	labels.append(content[:4].replace("\t", ""))
file.close()
sms=[s.replace("\t", "") for s in sms]

Generate TF-IDF Values

In [41]:
ps=PorterStemmer()
stopWords=set(stopwords.words("english"))
idf={}
tfidf_temp=[]
tfidf=[]
for sm in sms:
    words=word_tokenize(sm)
    wc=0
    tf={}
    for word in words:
        word=ps.stem(word)
        if word in stopWords:
            continue
        wc+=1
        if word in tf:
            tf[word]+=1
        else:
            tf[word]=1
    for key in tf:
        tf[key]/=wc
        if key in idf:
            idf[key]+=1
        else:
            idf[key]=1
    tfidf_temp.append(tf)
for key in idf:
    idf[key]=math.log(len(sms)/idf[key])

for x in tfidf_temp:
    for key in idf:
        if key in x:
            x[key]*=idf[key]
        else:
            x[key]=0
    myKeys = list(x.keys())
    myKeys.sort()
    sorted_dict = {i: x[i] for i in myKeys}
    tfidf.append(sorted_dict)


In [42]:
tfidf_matrix=[]
for x in tfidf:
    mat_row=[]
    for key in x:
        mat_row.append(x[key])
    tfidf_matrix.append(mat_row)
file=open("TF-IDF Matrix.txt", 'w')
for items in tfidf_matrix:
    for item in items:
        stringer=str(item)+" "
        file.write(stringer)
    file.write("\n")
file.close()
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, labels, random_state=109, test_size=0.1, shuffle=True)
print(len(X_train))
print(len(X_test))

5016
558


KNN Classifier

In [43]:
knn=KNeighborsClassifier(n_neighbors=4)
knn.fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.9247311827956989


K-Means Classifier

In [44]:
def kmeans(data, k, max_iterations=100):
    # Randomly initialize the cluster centers
    centroids = data[np.random.choice(data.shape[0], k, replace=False)]

    for _ in range(max_iterations):
        # Assign each data point to the nearest centroid
        distances = np.linalg.norm(data[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)

        # Update the centroids by taking the mean of all points assigned to each centroid
        new_centroids = np.array([data[labels == i].mean(axis=0) for i in range(k)])

        # Check for convergence
        if np.all(centroids == new_centroids):
            break

        centroids = new_centroids

    return centroids, labels

In [45]:
k = 2
np_matrix=np.array([np.array(xi) for xi in tfidf_matrix])
centroids, labelss = kmeans(np_matrix, k)
print("Final centroids:")
print(centroids)
c1=0
c2=0
count=0
for lab in labelss:
    if lab==0:
        c1+=1
    else:
        c2+=1
if(c1>c2):
    for i in range(0, len(tfidf_matrix)):
        if labelss[i]==0 and labels[i]=="ham":
            count+=1
        elif labelss[i]==1 and labels[i]=="spam":
            count+=1
else:
    for i in range(0, len(tfidf_matrix)):
        if labelss[i]==1 and labels[i]=="ham":
            count+=1
        elif labelss[i]==0 and labels[i]=="spam":
            count+=1
print("Accuracy of K-Means = ", (count*100/len(tfidf_matrix)))

Final centroids:
[[0.02976892 0.00811047 0.00108374 ... 0.00179904 0.0009198  0.00030348]
 [0.02165719 0.         0.         ... 0.         0.         0.        ]]
Accuracy of K-Means =  85.2529601722282
